# Optimization for Liwei and Danilo

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from summit.data import DataSet
from summit.domain import ContinuousVariable, Constraint, Domain
from summit.strategies import TSEMO2
from summit.models import GPyModel, AnalyticalModel
from summit.utils import pareto_efficient

from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from functools import partial


import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.art3d as art3d
import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm_notebook

## 1. Set up problem

Upload a CSV file with existing experimental data.

In [3]:
#TODO: File Upload
#TODO: Input columns
# Output columns

#Read in and clean up data
data_pd = pd.read_excel('data.xlsx')
input_columns = [ 'Texapon', 'DehytonAB30', 'Plantacare818', 'CC7BZ', 'ArlyponTT']
output_columns = ['viscosity', 'price', 'turbidity']
data_pd = data_pd[input_columns + output_columns]
data = DataSet.from_df(data_pd)
data.tail(5)

,Texapon,DehytonAB30,Plantacare818,CC7BZ,ArlyponTT,viscosity,price,turbidity
147,1.7759,8.1682,5.0536,1.6331,0.3188,0.993117,0.125094,0.005372
148,1.9610,7.9659,5.0708,1.0120,1.9342,0.911507,0.132936,0.218461
149,0.3214,9.8575,4.8186,0.5014,0.2759,0.981923,0.104258,0.001544
150,10.2248,0.0468,4.7296,0.1149,0.7092,0.995267,0.174555,0.002156
151,1.0205,0.4106,13.5673,0.3842,1.7966,0.992930,0.118646,0.535889


In [4]:
#TODO: Create Domain
#Set up the optimization problem domain
domain = Domain()

#Decision variables
domain += ContinuousVariable('Texapon', 
                             description = '',
                             bounds=[0, 15])
domain += ContinuousVariable('DehytonAB30', 
                             description = '',
                             bounds=[0, 15])
domain += ContinuousVariable('Plantacare818', 
                             description = '',
                             bounds=[0, 15])
domain += ContinuousVariable('CC7BZ', 
                             description = '',
                             bounds=[0, 2])
domain += ContinuousVariable('ArlyponTT', 
                             description = '',
                             bounds=[0, 2])

#Objectives
domain += ContinuousVariable('viscosity', 
                             description = '',
                             bounds=[0, 1],
                             is_objective=True)
# domain += ContinuousVariable('price', 
#                              description = '',
#                              bounds=[0, 1],
#                              is_objective=True)
# domain += ContinuousVariable('turbidity', 
#                              description = '',
#                              bounds=[0, 1],
#                              is_objective=True)

#Constraints
domain += Constraint('Texapon + DehytonAB30 + Plantacare818-15') #Make constraints of form <= 0

domain

Name,Type,Description,Values
Texapon,"continuous, input",,"[0,15]"
DehytonAB30,"continuous, input",,"[0,15]"
Plantacare818,"continuous, input",,"[0,15]"
CC7BZ,"continuous, input",,"[0,2]"
ArlyponTT,"continuous, input",,"[0,2]"
viscosity,"continuous, maximize objective",,"[0,1]"
,constraint,Texapon + DehytonAB30 + Plantacare818-15,


In [5]:
#price function
def price_function(X):
    price = 135.13*X['Texapon']+63.95*X['DehytonAB30']+62.87*X['Plantacare818']+ \
            90*X['CC7BZ']+75*X['ArlyponTT']
    price = price/1e4
    return np.atleast_2d(price.to_numpy()).T

## 2. Visualize Data

Let's visualize the data now.  Here we show the approximate pareto front based on existing experimental data. It has been difficult to achieve low values of viscosity.

In [6]:
button_visualize = widgets.Button(
    description='Visualize Data',
    tooltip='Visualize in a 3D plot',
    icon='play')

output_visualize = widgets.Output()
display(button_visualize, output_visualize)

def visualize_data(b):
    _ = plt.tight_layout()
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    pareto_front, indices = pareto_efficient(data[['viscosity', 'price', 'turbidity']].to_numpy(),
                                      maximize=False)
    # ax.scatter(data['viscosity'], data['price'], data['turbidity'], 
    #            alpha=0.1, label='all data', marker='^', s=50)
    img = ax.scatter(pareto_front[:, 0], pareto_front[:, 1], pareto_front[:, 2], 
                     alpha=0.7, label='pareto front', s=100, c=data.index[indices])
    _ = plt.colorbar(img, label='Experiment number')
    ax.set_xlabel('viscosity'); ax.set_ylabel('price'); ax.set_zlabel('turbidity')
    for xi, yi, zi in pareto_front:        
        line=art3d.Line3D(*zip((xi, yi, 0), (xi, yi, zi)), 
                          marker='o', markevery=(1, 1), c='k',alpha=0.5)
        _ = ax.add_line(line)
    _ = ax.view_init(20, -60)
    _ = ax.set_title('Approximate Pareto Front')
    output_visualize.clear_output(wait=True)
    with output_visualize:
        display(fig)
button_visualize.on_click(visualize_data)

Button(description='Visualize Data', icon='play', style=ButtonStyle(), tooltip='Visualize in a 3D plot')

Output()

## 3. Run Optimization

Now, we can run the optimization. Click the button that says run optimization.  It might take a couple minutes.

In [7]:

    
num_experiments = widgets.BoundedIntText(
    value=3,
    min=0,
    max=10,
    step=1,
    description='# Expr:',
    disabled=False
)
button_run_opt = widgets.Button(
    description='Run optimization',
    tooltip='Run the optimization',
    icon='play')
output_run_opt = widgets.Output()

display(num_experiments, button_run_opt, output_run_opt)

input_dim = domain.num_continuous_dimensions() + domain.num_discrete_variables()
# models = {'viscosity': GPyModel(input_dim=input_dim), 
#           'price': AnalyticalModel(function=price_function),
#           'turbidity': GPyModel(input_dim=input_dim)
#          }
models = {'viscosity': GPyModel(input_dim=input_dim)}
tsemo = TSEMO2(domain, models)

def run_optimization(b):
    n = num_experiments.value

    with output_run_opt:
        print("Starting. This might take a couple minutes.")
        
    experiments = tsemo.generate_experiments(data, n)
    output_run_opt.clear_output()
    with output_run_opt:
        print("Next experiments:")
        display(experiments)

button_run_opt.on_click(run_optimization)


BoundedIntText(value=3, description='# Expr:', max=10)

Button(description='Run optimization', icon='play', style=ButtonStyle(), tooltip='Run the optimization')

Output()

In [ ]:
tsemo.generate_experiments(data, 3)

> /Users/Kobi/Documents/Research/summit/summit/strategies.py(192)select_max_hvi()
    191             import ipdb; ipdb.set_trace()
--> 192             hvY = HvI.hypervolume(Yfront, [0, 0])
    193             #Determine hypervolume improvement by including



ipdb>  Yfront


array([[-0.0026]])


ipdb>  q


--KeyboardInterrupt--


ipdb>  exit


## 4. Validate Model

We'll make a cross validation plot next. Click on the button to generate one.

In [9]:
button_plot_cv = widgets.Button(
    description='Make CV Plot',
    tooltip='CV Plot',
    icon='play')
output_plot_cv = widgets.Output()
display(button_plot_cv, output_plot_cv)
def plot_cv(b):
    gp = GPyModel(input_dim=input_dim)
    X = data[input_columns].to_numpy(dtype=np.float64)
    Y = data[output_columns].to_numpy(dtype=np.float64)

    #Mask out weird data
    mask = np.ones(X.shape[0],dtype=bool)
    indices = np.where(Y[:, 0]>1.0)[0]
    mask[indices] = 0
    # Y = Y[mask, :]
    X = X[mask, :]

    
    with output_plot_cv:
        print('Making plots...')
        
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    for i, name in enumerate(['viscosity', 'turbidity']):
        kf = KFold(n_splits=3)
        scores = np.zeros(5)

        Y = data[name].to_numpy(dtype=np.float64)
        Y = Y[mask]
        for train, test in kf.split(X):
            gp.fit(X[train], np.atleast_2d(Y[train]).T)
            y_predict = gp.predict(X[test])
            axes[i].scatter(Y[test], y_predict, c='k')
        min_y = Y.min()
        max_y = Y.max()
        axes[i].plot([min_y, max_y], [min_y, max_y], 'k--', lw=4)
        axes[i].set_xlabel('Measured')
        axes[i].set_ylabel('Predicted')
        axes[i].set_title(f'{name} Cross Validation'.title())
    output_plot_cv.clear_output(wait=True)
    with output_plot_cv:
        plt.show()
button_plot_cv.on_click(plot_cv)

Button(description='Make CV Plot', icon='play', style=ButtonStyle(), tooltip='CV Plot')

Output()